In [120]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from typing import Literal, List
import pandas as pd
from tqdm import tqdm

In [ ]:
api_key = '...'

In [122]:
llm = ChatOpenAI(
	base_url='https://api.proxyapi.ru/openai/v1',
	api_key=api_key,
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2)

In [123]:
with open('../prompts/mistral_prompt.txt', 'r') as f:
	template = f.read()
prompt = PromptTemplate.from_template(template)
class LLMResponse(BaseModel):
    """Рассуждения и класс"""
    index: int
    sample: str
    reasoning: str = Field(description="Рассуждения перед ответа")
    label: Literal[
        "SERVICE",
        "NON_FOOD_GOODS",
        "LOAN",
        "NOT_CLASSIFIED",
        "LEASING",
        "FOOD_GOODS",
        "BANK_SERVICE",
        "TAX",
        "REALE_STATE"
    ] = Field(description="Ответ модели, класс для запроса")

class ListResponses(BaseModel):
    answers: List[LLMResponse]

structured_llm = llm.with_structured_output(ListResponses, include_raw=True)
chain = prompt | structured_llm

In [124]:
def func(row):
	return str(row['id']) + '. ' + row['content']

In [125]:
data = pd.read_csv('../data/payments_main.tsv', sep='\t', names=['id', 'date', 'price', 'content'])
data['content'] = data.apply(func, axis=1)

### GroupInference

In [134]:
keys = ['question1', 'question2', 'question3', 'question4', 'question5']
llm_responses = []
for i in tqdm(range(0, 10000, 5)):
	sample = data.loc[i:(i+5), :]
	content = sample['content']
	mapper  = dict(zip(keys, content.tolist()))
	answer = chain.invoke(mapper)
	if answer['parsing_error'] is None:
		answers_list = answer['parsed']
		llm_responses.extend(answers_list.answers)
	else:
		print(f"Error {answer['parsing_error']}")

100%|██████████| 2000/2000 [3:51:35<00:00,  6.95s/it]  


In [140]:
len(llm_responses)

9996

In [144]:
llm_responses[0].dict()

{'index': 1,
 'sample': 'За тур.поездку по договору №001 от 27.01.2023г',
 'reasoning': 'Описание платежа связано с туристической поездкой, что может подразумевать оплату услуг, связанных с путешествием. Однако, конкретные слова, указывающие на услуги, отсутствуют, и это может быть не совсем однозначно. Поэтому, учитывая, что это не попадает под другие классы, я отнесу это к классу NOT_CLASSIFIED.',
 'label': 'NOT_CLASSIFIED'}

In [145]:
responses_json = [response.dict() for response in llm_responses]

In [152]:
import json
with open("4o-mini_labels.json", "w") as f:
	json.dump(responses_json, f)

In [153]:
with open("4o-mini_labels.json", "r") as f:
	labels = json.load(f)